In [8]:
%matplotlib inline
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from scipy import io
import pickle

# Own code
sys.path.append("../")
from utils.data_utils import generate_dgp_tvp_var, generate_matrices, generate_contemp_matrices
from utils.tvp_models import TVPVARModel, tvp_ar_contemp, tvp_ar_non_contemp

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

In [9]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    prior_train = train-10
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [10]:
%%time
import time
import os
from multiprocessing import Pool, Array

# They are going to be some disgusting warnings for the first iterations of the model (has to do with initialization)
import warnings
warnings.filterwarnings("ignore")

np.random.seed(12345)
n_iterations = 200

iterations_set = np.arange(1,n_iterations+1,1)
M = (2,5)
prior_list = ["svss"]#, "lasso_alternative", "horseshoe"]
args_tuples = [x for sub_list in [x for sub_list in [[[(n, m, prior) for n in iterations_set] for m in M] for prior in prior_list] for x in sub_list] for x in sub_list]

if __name__ == '__main__':
    
    pool = Pool()
    simulation_results = pool.starmap(simulation_run, args_tuples)

OSError: ../simulations/datasets/y_2_200_1_0.4_8_het_py.csv not found.

In [ ]:
T = 200
train = 175
p = 1
sparsity = 0.40
prior = "svss"

simulation_parameters = [T,p,train, 1/2, 1/4, 4*1e-5, 1, 1/2, 1e-1*2, 1e-2, 1e-9, 12345]

dump_to_disk = [simulation_results, simulation_parameters]

with open(f'../simulations/results/statistics_{T}_{p}_{n_iterations}_{prior}_{sparsity}.pkl', 'wb') as f:
        pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
optimal_set = []

for run in range(1,11):

    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    prior_train = train-10
    sparsity = 0.05
    M = 2
    prior = "lasso_alternative"

    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")

    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)

    prior_parameters = None

    a0_set = np.linspace(1,5,5)
    b0_set = np.hstack(([1], np.linspace(3,12,4)))
    tpl_list = [x for sub_list in [[(a,b) for a in a0_set] for b in b0_set] for x in sub_list]

    msfe_list = []

    for parameters in tpl_list:

        optim_pior_parameters = {'a0_lasso':parameters[0],'b0_lasso':parameters[1]}
        msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(train, M, p, prior_train, X_matrix_contemp[:train], y_matrix_contemp[:,:train], prior, print_status=False, iterations=100, prior_parameters=optim_pior_parameters)

        msfe_list.append(msfe_contemp.mean())

    optimal_prior = tpl_list[msfe_list.index(min(msfe_list))]
    prior_parameters = {'a0_lasso':optimal_prior[0],'b0_lasso':optimal_prior[1]}

    print(f"Run: {run} -> Optimal prior: {optimal_prior}")
    optimal_set.append(optimal_prior)
    

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    prior_train = train-10
    sparsity = 0.05
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
#     prior_parameters = None
    
#     if prior == 'lasso_alternative':
        
#         a0_set = np.linspace(1,5,5)
#         b0_set = np.hstack(([1], np.linspace(3,12,4)))
#         tpl_list = [x for sub_list in [[(a,b) for a in a0_set] for b in b0_set] for x in sub_list]
        
#         msfe_list = []
        
#         for parameters in tpl_list:
            
#             optim_pior_parameters = {'a0_lasso':parameters[0],'b0_lasso':parameters[1]}
#             msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(train, M, p, prior_train, X_matrix_contemp[:train], y_matrix_contemp[:,:train], prior, print_status=False, iterations=100, prior_parameters=optim_pior_parameters)
            
#             msfe_list.append(msfe_contemp.mean())
        
#         optimal_prior = tpl_list[msfe_list.index(min(msfe_list))]
#         prior_parameters = {'a0_lasso':optimal_prior[0],'b0_lasso':optimal_prior[1]}
        
#     elif prior == 'svss':
        
#         g0_set = np.linspace(1,5,5)
#         h0_set = np.hstack(([1], np.linspace(3,12,4)))
#         tpl_list = [x for sub_list in [[(g,h) for g in g0_set] for h in h0_set] for x in sub_list]
        
#         msfe_list = []
        
#         for parameters in tpl_list:
            
#             optim_pior_parameters = {'g0':parameters[0],'h0':parameters[1], 'pi0': 0.5}
#             msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(train, M, p, prior_train, X_matrix_contemp[:train], y_matrix_contemp[:,:train], prior, print_status=False, iterations=100, prior_parameters=optim_pior_parameters)
            
#             msfe_list.append(msfe_contemp.mean())
        
#         optimal_prior = tpl_list[msfe_list.index(min(msfe_list))]
#         prior_parameters = {'g0': optimal_prior[0], 'h0': optimal_prior[1], 'pi0': 0.5}
        
#     elif prior == 'horseshoe':
#         prior_parameters = {'a0': 1, 'b0': 1}
        
#         a0_set = np.hstack(([1.5], np.linspace(3,12,4)))
#         b0_set = np.hstack(([1.5],np.linspace(1,5,5)))
#         tpl_list = [x for sub_list in [[(a,b) for a in a0_set] for b in b0_set] for x in sub_list]
        
#         msfe_list = []
        
#         for parameters in tpl_list:
            
#             optim_pior_parameters = {'a0':parameters[0],'b0':parameters[1]}
#             msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(train, M, p, prior_train, X_matrix_contemp[:train], y_matrix_contemp[:,:train], prior, print_status=False, iterations=100, prior_parameters=optim_pior_parameters)
            
#             msfe_list.append(msfe_contemp.mean())
        
#         optimal_prior = tpl_list[msfe_list.index(min(msfe_list))]
#         prior_parameters = {'a0':optimal_prior[0],'b0':optimal_prior[1]}
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]